# Setup

In [1]:
from langchain_openai import OpenAI
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
import yaml
import os

In [2]:
with open('config.yaml', 'r') as config_file:
    config = yaml.safe_load(config_file)
os.environ['OPENAI_API_KEY'] = config['OPENAI_API_KEY']

In [3]:
openai = OpenAI(model_name='gpt-3.5-turbo-instruct', temperature=0)

# RouterChain

In [4]:
# ela inicial (classificação)
chain = (
    PromptTemplate.from_template(
        """
        Classifique a pergunta do usuário em uma das seguintes categorias:
        - Assuntos Financeiros
        - Suporte Técnico
        - Atualização de Cadastro
        - Outras Informações

        Pergunta: {query}
        Classificação:
        """
    )
    | openai
    | StrOutputParser() 
)

In [5]:
print(chain)

first=PromptTemplate(input_variables=['query'], template='\n        Classifique a pergunta do usuário em uma das seguintes categorias:\n        - Assuntos Financeiros\n        - Suporte Técnico\n        - Atualização de Cadastro\n        - Outras Informações\n\n        Pergunta: {query}\n        Classificação:\n        ') middle=[OpenAI(client=<openai.resources.completions.Completions object at 0x72d0feebb950>, async_client=<openai.resources.completions.AsyncCompletions object at 0x72d0feed0190>, temperature=0.0, openai_api_key=SecretStr('**********'), openai_proxy='')] last=StrOutputParser()


In [6]:
# elos específicos
financial_chain = PromptTemplate.from_template(
    """
    Você é um especialista financeiro.
    Sempre responda às perguntas começando com "Bem-vindo ao Suporte Financeiro".
    Responda à pergunta do usuário:
    Pergunta: {query}
    Resposta:
    """
) | openai
tech_support_chain = PromptTemplate.from_template(
    """
    Você é um especialista em suporte técnico.
    Sempre responda às perguntas começando com "Bem-vindo ao Suporte Técnico".
    Ajude o usuário com seu problema técnico.
    Pergunta: {query}
    Resposta:
    """
) | openai
update_registration_chain = PromptTemplate.from_template(
    """
    Você é um representante de atendimento ao cliente.
    Sempre responda às perguntas começando com "Bem-vindo ao Suporte de Cadastro".
    Guie o usuário na atualização de suas informações de cadastro.
    Pergunta: {query}
    Resposta:
    """
) | openai
other_info_chain = PromptTemplate.from_template(
    """
    Você é um assistente de informações gerais.
    Sempre responda às perguntas começando com "Bem-vindo ao Suporte Geral".
    Forneça informações ao usuário sobre sua pergunta.
    Pergunta: {query}
    Resposta:
    """
) | openai

In [7]:
# Função de roteamento
def route(info):
    topic = info["topic"].lower()
    if "financeiro" in topic:
        return financial_chain
    elif "técnico" in topic:
        return tech_support_chain
    elif "atualização" in topic or "cadastro" in topic:
        return update_registration_chain
    else:
        return other_info_chain

In [8]:
# Exemplos 1 suporte técnico
classification = chain.invoke({"query": "Como faço para redefinir minha senha?"})
print(classification)

 Suporte Técnico


In [9]:
#chama a função rote, passando o topico
response_chain = route({"topic": classification})
print(response_chain)

first=PromptTemplate(input_variables=['query'], template='\n    Você é um especialista em suporte técnico.\n    Sempre responda às perguntas começando com "Bem-vindo ao Suporte Técnico".\n    Ajude o usuário com seu problema técnico.\n    Pergunta: {query}\n    Resposta:\n    ') last=OpenAI(client=<openai.resources.completions.Completions object at 0x72d0feebb950>, async_client=<openai.resources.completions.AsyncCompletions object at 0x72d0feed0190>, temperature=0.0, openai_api_key=SecretStr('**********'), openai_proxy='')


In [10]:
#executa o objeto correto
response = response_chain.invoke({"query": "Como faço para redefinir minha senha?"})
print(response)

Bem-vindo ao Suporte Técnico. Para redefinir sua senha, você precisará acessar a página de login e clicar na opção "Esqueci minha senha". Em seguida, siga as instruções na tela para redefinir sua senha. Se você tiver problemas durante o processo, não hesite em entrar em contato conosco novamente para obter ajuda adicional.


In [11]:
# Exemplo 2 (Assuntos Financeiros)
classification = chain.invoke({"query": "Como posso pagar uma fatura atrasada?"})
response_chain = route({"topic": classification})
response = response_chain.invoke({"query": "Como posso pagar uma fatura atrasada?"})
print(response)

Bem-vindo ao Suporte Financeiro. Como posso ajudá-lo?
    Para pagar uma fatura atrasada, você pode acessar o seu perfil no nosso site e selecionar a opção de pagamento. Lá, você poderá escolher a forma de pagamento e inserir os dados necessários para efetuar o pagamento. Se preferir, também é possível entrar em contato com a nossa central de atendimento e realizar o pagamento por telefone.


In [12]:
# Exemplo 3 (Atualização de Cadastro)
classification = chain.invoke({"query": "Preciso alterar meu endereço de e-mail."})
response_chain = route({"topic": classification})
response = response_chain.invoke({"query": "Preciso alterar meu endereço de e-mail."})
print(response)

Bem-vindo ao Suporte de Cadastro. Para alterar seu endereço de e-mail, siga os seguintes passos:
    1. Acesse o site da nossa empresa e faça login na sua conta.
    2. Clique na opção "Minha Conta" ou "Configurações".
    3. Procure pela opção "Informações de Contato" ou "Dados Pessoais".
    4. Clique em "Editar" ou no ícone de lápis ao lado do seu endereço de e-mail atual.
    5. Insira o novo endereço de e-mail desejado e clique em "Salvar".
    Pronto! Seu endereço de e-mail foi atualizado com sucesso. Lembre-se de utilizar esse novo endereço para fazer login na próxima vez que acessar nossa plataforma.


In [13]:
# Exemplo 4 (Outras Informações)
classification = chain.invoke({"query": "Qual é a missão da empresa?"})
response_chain = route({"topic": classification})
response = response_chain.invoke({"query": "Qual é a missão da empresa?"})
print(response)

Bem-vindo ao Suporte Geral. A missão da empresa é fornecer produtos e serviços de alta qualidade para atender às necessidades de nossos clientes e contribuir para o crescimento e sucesso de suas empresas. Nós nos esforçamos para ser uma empresa líder em inovação, sustentabilidade e responsabilidade social.
